Random Forest Regression (Kristin)
1.  Model #1: default parameters
2.  Model #2:  add max_sample_leafs
3.  Model #3:  increase n_estimators and decrease max_sample_leafs




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns



vgsales = pd.read_excel('vgsales_v5.xlsx')

vgsales.head()

# Create dummy variables for | Genre | First_Party | Platform_Type

#dummy variables for Genre
dummy_genre = pd.get_dummies(vgsales.Genre)
dummy_genre

#dummy variables for First Party
dummy_first_party = pd.get_dummies(vgsales.First_Party)
dummy_first_party

#dummy variables for Platform Type
dummy_platform_type = pd.get_dummies(vgsales.Platform_Type)
dummy_platform_type

#Merge dummy frames into one dataset with original vgsales datframe
merged_vgsales = pd.concat([vgsales, dummy_genre, dummy_first_party, dummy_platform_type], axis='columns')






In [2]:
vgsales_df = pd.DataFrame(merged_vgsales)



In [3]:
#drop Rank (it is perfectly correlated with sales because it is based on global sales & is a result of sales
# drop Name (a significant number of dummy variables would have to be created and we are not analyzing the impact of name, but trying to understand impact of genre, platform and region on sales)
#drop Platform (we have created Platform Type and First Parties categories that are more meaningful than Platform)
#drop Publisher (similar reason as Name)

vgsales_df2 = vgsales_df.drop(columns = ["Rank","Name", "Platform", "Publisher","Genre", "First_Party", "Platform_Type","Year_1_Sales", "Year", "Atari","SEGA", "Other"])

In [4]:
#Create X, y

X = vgsales_df2.drop(columns = "Global_Sales")
y = vgsales_df2.Global_Sales

In [5]:
#create test/train/validation sets, 80% = train, 10% = test, 10% = validation
#check shape to ensure similar sizes 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(12783, 24)
(12783,)
(1598, 24)
(1598,)
(1598, 24)
(1598,)


Random Forest

In [6]:
#import libraries for random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import timeit
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score



In [7]:
#Model #1:  using default parameters, except i set n_estimators to 100 to avoid pink error

rnd_reg1 = RandomForestRegressor(n_estimators = 100)
rnd_reg1.fit(X_train, y_train)

y_pred_rf1 = rnd_reg1.predict(X_val)
print("mse:",mean_squared_error(y_val, y_pred_rf1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred_rf1)))
print("r-squared:", r2_score(y_val, y_pred_rf1))
print("training time:", timeit.timeit())

#good results, low rmse and hight r-squares


mse: 0.07703087593867343
rmse: 0.27754436751386874
r-squared: 0.962614931388856
training time: 0.008365099999998904


In [8]:
#Model #2:  keep n_estimators at 100, set max_leaf_nodes to 16

rnd_reg2 = RandomForestRegressor(n_estimators = 100, max_leaf_nodes = 16)
rnd_reg2.fit(X_train, y_train)

y_pred_rf2 = rnd_reg2.predict(X_val)
print("mse:",mean_squared_error(y_val, y_pred_rf2))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred_rf2)))
print("r-squared:", r2_score(y_val, y_pred_rf2))
print("training time:", timeit.timeit())

#good results, but not as good as #1

mse: 0.14491438981596322
rmse: 0.380676227017085
r-squared: 0.9296693132462236
training time: 0.00779339999999884


In [9]:
#Model #3:   n_estimators at 500 and max_leaf_nodes to 5

rnd_reg3 = RandomForestRegressor(n_estimators = 500, max_leaf_nodes = 5)
rnd_reg3.fit(X_train, y_train)

y_pred_rf3 = rnd_reg3.predict(X_val)
print("mse:",mean_squared_error(y_val, y_pred_rf3))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred_rf3)))
print("r-squared:", r2_score(y_val, y_pred_rf3))
print("training time:", timeit.timeit())

#slower than other models and the worst results, stick with model 1, default parameters with n_estimators set to 100

mse: 0.31576435324966223
rmse: 0.561929135434053
r-squared: 0.8467514244471225
training time: 0.010345100000000329


In [14]:
#run random forest based on article https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e

rf4 = RandomForestRegressor(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42)
rf4.fit(X_train, y_train)

print('R^2 Training Score: {:.2f} \nOOB Score: {:.2f} \nR^2 Validation Score: {:.2f}'.format(rf4.score(X_train, y_train), 
                                                                                             rf4.oob_score_,
                                                                                             rf4.score(X_val, y_val)))
y_pred_rf4 = rf4.predict(X_val)
print("mse:",mean_squared_error(y_val, y_pred_rf4))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred_rf4)))
print("r-squared:", r2_score(y_val, y_pred_rf4))
print("training time:", timeit.timeit())


R^2 Training Score: 1.00 
OOB Score: 0.97 
R^2 Validation Score: 0.97
mse: 0.06201651093241559
rmse: 0.2490311445028826
r-squared: 0.9699017895359526
training time: 0.03561089999999467


In [11]:
#permutation importances
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(rf4, X_train, y_train):
    return r2_score(y_train, rf4.predict(X_train))

perm_imp_rfpimp = permutation_importances(rf4, X_train, y_train, r2)

perm_imp_rfpimp


,Importance
Feature,
NA_Sales,8.646040e-01
EU_Sales,1.116441e-01
JP_Sales,4.457265e-02
Avg_Annual_Sales,3.922871e-02
Other_Sales,6.213028e-03
Years_In_Market,4.926145e-03
Nintendo,7.855709e-04
Console,1.023527e-04
Sony,7.687196e-05


In [12]:
#the example in book uses classification, need to change the below to regression
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensembles import ExtraTreesRegressor

labels = vgsales_df2.drop(columns = ["Global_Sales"])

# Build a classification task using 3 informative features
X_train, y_train = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=3,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=0,
                           shuffle=False)

# Build a forest and compute the impurity-based feature importances
forest = ExtraTreesRegressor(n_estimators=250,
                              random_state=0)

forest.fit(X_train, y_train)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

Automatically created module for IPython interactive environment


ModuleNotFoundError: No module named 'sklearn.ensembles'